In [21]:
#Блок в рамках которого импотряться математические подсчеты, падас и нампай
import pandas as pd
import numpy as np
from scipy.stats import iqr
from scipy import stats
from scipy.stats import skew
from scipy.stats import kurtosis
import scipy.stats
import scipy as sp
import scipy.stats as stats
from scipy.stats import t

#Блок в рамках которого импотряться графики
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns 
from matplotlib.pyplot import figure
sns.set_style('darkgrid')

#Блок в рамках которого импотряться статистические пакеты
import statsmodels.api as sm 
import statsmodels.formula.api as smf 
from sklearn.preprocessing import OneHotEncoder
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from statsmodels.regression.linear_model import OLS
from sklearn.decomposition import PCA
from statsmodels.stats.diagnostic import het_white, het_goldfeldquandt, het_breuschpagan
from statsmodels.stats.outliers_influence import reset_ramsey
from statsmodels.stats.diagnostic import compare_encompassing 
from linearmodels.iv import IV2SLS

#Блок в рамках которого импотряться операционные пакеты
import os


# Содержание работы
7. [Endogeneity](#subparagraph0)  - решение проблемы эндогенности

In [22]:
cd "C:\Users\Alexandr\Documents\GitHub\smartphone-price-analysis\"

C:\Users\Alexandr\Documents\GitHub\smartphone-price-analysis


In [23]:
data = pd.read_csv('data\PhonesDF_years_corrected_v2.csv').set_index('phone_title')
all_spec = pd.read_csv('data\PhonesPriceInKenya_v2.csv').set_index('Phone Title')['Specs Score']
data.shape

(559, 32)

In [24]:
data = data.join(pd.DataFrame(all_spec))
data['Specs Score'] = data['Specs Score'].str.replace('%', '').astype(int)
data = add_constant(data)
data.shape

(559, 34)

In [25]:
data.isna().sum().sum()

0

### 7. Endogeneity

В рамках датасета можно отметить, что пропущены характеристики, отражающие производительность смартфона (графический и центральный процессоры). В то же время такие характеристики как год выпуска ('year_of_release'), оперативная память ('ram'), ёмкость батареи ('battery_capacity') могут быть иметь коореляцию с мощностью графического и/или центрального процессора. Следовательно, это может приводить к к проблеме эндогенности.

В рамках решения данной проблемы воспользуюмся 2SLS

#### 7.1 year_of_release - эндогенный

In [32]:
mask_exog_and_instr1 = ['const', 'rating', 'disp_diag', 'camera', 'storage',
               'density', 'Other_oper_syst', 'battery_type_LiPo', 'brand_Apple',
               'brand_BlackBerry', 'brand_Cubot', 'brand_Gionee', 'brand_Google',
               'brand_HTC', 'brand_Honor', 'brand_Huawei', 'brand_Infinix', 'brand_LG',
               'brand_Lava', 'brand_Lenovo', 'brand_Motorola', 'brand_Nokia',
               'brand_OnePlus', 'brand_Oppo', 'brand_Realme', 'brand_Samsung',
               'brand_Sony', 'brand_Tecno', 'brand_Vivo', 'Specs Score', 'ram', 'battery_capacity']

mask_exog1 = ['const', 'rating', 'disp_diag', 'camera', 'storage',
               'density', 'Other_oper_syst', 'battery_type_LiPo', 'brand_Apple',
               'brand_BlackBerry', 'brand_Cubot', 'brand_Gionee', 'brand_Google',
               'brand_HTC', 'brand_Honor', 'brand_Huawei', 'brand_Infinix', 'brand_LG',
               'brand_Lava', 'brand_Lenovo', 'brand_Motorola', 'brand_Nokia',
               'brand_OnePlus', 'brand_Oppo', 'brand_Realme', 'brand_Samsung',
               'brand_Sony', 'brand_Tecno', 'brand_Vivo', 'ram', 'battery_capacity']

In [33]:
first_step_2SLS1 = IV2SLS(dependent = data['year_of_release'],  exog = data[mask_exog_and_instr1], endog = None, instruments= None, weights=  None)
first_step_2SLS1 = first_step_2SLS1.fit()
first_step_2SLS1

OLS Estimation Summary                            
==============================================================================
Dep. Variable:        year_of_release   R-squared:                      0.6531
Estimator:                        OLS   Adj. R-squared:                 0.6327
No. Observations:                 559   F-statistic:                    1986.6
Date:                Tue, May 09 2023   P-value (F-stat)                0.0000
Time:                        23:05:28   Distribution:                 chi2(31)
Cov. Estimator:                robust                                         
                                                                              
                                 Parameter Estimates                                 
=====================================================================================
                   Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
-------------------------------------------------------------------------------------
const                 2010.2     0.7666     2622.2     0.0000      2008.7      2011.7
rating                0.1933     0.0651     2.9670     0.0030      0.0656      0.3210
disp_diag             1.0263     0.1632     6.2897     0.0000      0.7065      1.3462
camera               -0.0016     0.0058    -0.2767     0.7820     -0.0130      0.0098
storage               0.0017     0.0008     2.1490     0.0316      0.0002      0.0033
density           -1.505e-05  2.524e-06    -5.9604     0.0000  -1.999e-05   -1.01e-05
Other_oper_syst      -2.3844     0.5697    -4.1854     0.0000     -3.5009     -1.2678
battery_type_LiPo     0.2980     0.1146     2.5997     0.0093      0.0733      0.5226
brand_Apple           0.3122     0.3836     0.8138     0.4158     -0.4397      1.0640
brand_BlackBerry      0.7871     0.4045     1.9458     0.0517     -0.0057      1.5799
brand_Cubot          -0.7718     0.2390    -3.2297     0.0012     -1.2402     -0.3034
brand_Gionee         -1.3753     0.1488    -9.2421     0.0000     -1.6669     -1.0836
brand_Google          0.5238     0.2858     1.8332     0.0668     -0.0362      1.0839
brand_HTC            -0.1606     0.2046    -0.7849     0.4325     -0.5617      0.2405
brand_Honor           0.6852     0.1745     3.9271     0.0001      0.3432      1.0272
brand_Huawei          0.1337     0.1312     1.0194     0.3080     -0.1234      0.3909
brand_Infinix         0.2548     0.1628     1.5655     0.1175     -0.0642      0.5738
brand_LG             -0.1627     0.3409    -0.4773     0.6331     -0.8308      0.5054
brand_Lava            2.5070     0.2341     10.708     0.0000      2.0481      2.9658
brand_Lenovo         -0.8641     0.4794    -1.8026     0.0715     -1.8037      0.0755
brand_Motorola        0.0381     0.2597     0.1468     0.8833     -0.4709      0.5471
brand_Nokia           1.3773     0.2371     5.8080     0.0000      0.9125      1.8420
brand_OnePlus        -0.2467     0.1862    -1.3251     0.1851     -0.6116      0.1182
brand_Oppo            0.2239     0.1977     1.1327     0.2573     -0.1635      0.6113
brand_Realme          0.4251     0.1210     3.5124     0.0004      0.1879      0.6623
brand_Samsung         0.4131     0.1309     3.1556     0.0016      0.1565      0.6697
brand_Sony            0.2440     0.3126     0.7805     0.4351     -0.3687      0.8568
brand_Tecno           0.2452     0.1851     1.3251     0.1851     -0.1175      0.6080
brand_Vivo           -0.0726     0.1577    -0.4604     0.6452     -0.3817      0.2365
Specs Score           0.0139     0.0047     2.9650     0.0030      0.0047      0.0231
ram                  -0.0110     0.0279    -0.3923     0.6948     -0.0657      0.0438
battery_capacity      0.0002  7.629e-05     2.3748     0.0176   3.165e-05      0.0003
=====================================================================================
OLSResults, id: 0x16a896859d0

In [35]:
res_iv1 = IV2SLS(data['log_price'], data[mask_exog1], data['year_of_release'], 
                data['Specs Score']).fit()
print(res_iv1)

                          IV-2SLS Estimation Summary                          
Dep. Variable:              log_price   R-squared:                      0.5741
Estimator:                    IV-2SLS   Adj. R-squared:                 0.5490
No. Observations:                 559   F-statistic:                    2748.6
Date:                Tue, May 09 2023   P-value (F-stat)                0.0000
Time:                        23:05:38   Distribution:                 chi2(31)
Cov. Estimator:                robust                                         
                                                                              
                                 Parameter Estimates                                 
                   Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
-------------------------------------------------------------------------------------
const                -116.81     302.07    -0.3867     0.6990     -708.86      475.23
rating                0.

In [36]:
print(res_iv1.wu_hausman())


Wu-Hausman test of exogeneity
H0: All endogenous variables are exogenous
Statistic: 0.0305
P-value: 0.8614
Distributed: F(1,526)


#### 7.2 ram - эндогенный

In [37]:
mask_exog_and_instr2 = ['const', 'rating', 'disp_diag', 'camera', 'storage',
               'density', 'Other_oper_syst', 'battery_type_LiPo', 'brand_Apple',
               'brand_BlackBerry', 'brand_Cubot', 'brand_Gionee', 'brand_Google',
               'brand_HTC', 'brand_Honor', 'brand_Huawei', 'brand_Infinix', 'brand_LG',
               'brand_Lava', 'brand_Lenovo', 'brand_Motorola', 'brand_Nokia',
               'brand_OnePlus', 'brand_Oppo', 'brand_Realme', 'brand_Samsung',
               'brand_Sony', 'brand_Tecno', 'brand_Vivo', 'Specs Score', 'year_of_release', 'battery_capacity']

mask_exog2 = ['const', 'rating', 'disp_diag', 'camera', 'storage',
               'density', 'Other_oper_syst', 'battery_type_LiPo', 'brand_Apple',
               'brand_BlackBerry', 'brand_Cubot', 'brand_Gionee', 'brand_Google',
               'brand_HTC', 'brand_Honor', 'brand_Huawei', 'brand_Infinix', 'brand_LG',
               'brand_Lava', 'brand_Lenovo', 'brand_Motorola', 'brand_Nokia',
               'brand_OnePlus', 'brand_Oppo', 'brand_Realme', 'brand_Samsung',
               'brand_Sony', 'brand_Tecno', 'brand_Vivo', 'year_of_release', 'battery_capacity']

In [38]:
first_step_2SLS2 = IV2SLS(dependent = data['ram'],  exog = data[mask_exog_and_instr2], endog = None, instruments= None, weights=  None)
first_step_2SLS2 = first_step_2SLS2.fit()
first_step_2SLS2

OLS Estimation Summary                            
==============================================================================
Dep. Variable:                    ram   R-squared:                      0.6312
Estimator:                        OLS   Adj. R-squared:                 0.6095
No. Observations:                 559   F-statistic:                    3296.7
Date:                Tue, May 09 2023   P-value (F-stat)                0.0000
Time:                        23:05:59   Distribution:                 chi2(31)
Cov. Estimator:                robust                                         
                                                                              
                                 Parameter Estimates                                 
=====================================================================================
                   Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
-------------------------------------------------------------------------------------
const                 65.322     170.43     0.3833     0.7015     -268.72      399.36
rating               -0.1841     0.1203    -1.5299     0.1260     -0.4199      0.0517
disp_diag             0.3183     0.2288     1.3914     0.1641     -0.1301      0.7668
camera                0.0188     0.0094     1.9987     0.0456      0.0004      0.0373
storage               0.0105     0.0015     7.1526     0.0000      0.0076      0.0134
density            -1.33e-06  3.669e-06    -0.3625     0.7170   -8.52e-06   5.861e-06
Other_oper_syst      -0.8224     0.4440    -1.8524     0.0640     -1.6925      0.0478
battery_type_LiPo    -0.4301     0.1962    -2.1917     0.0284     -0.8148     -0.0455
brand_Apple          -1.5589     0.4141    -3.7645     0.0002     -2.3705     -0.7473
brand_BlackBerry      0.7318     0.3687     1.9847     0.0472      0.0091      1.4544
brand_Cubot          -0.1154     0.4915    -0.2349     0.8143     -1.0788      0.8479
brand_Gionee          1.4295     0.5328     2.6829     0.0073      0.3852      2.4737
brand_Google          0.5493     0.3376     1.6269     0.1038     -0.1125      1.2111
brand_HTC             0.5480     0.5505     0.9954     0.3195     -0.5310      1.6271
brand_Honor           0.1417     0.3811     0.3717     0.7101     -0.6054      0.8887
brand_Huawei         -0.2177     0.1882    -1.1571     0.2472     -0.5866      0.1511
brand_Infinix        -0.1762     0.2082    -0.8464     0.3973     -0.5842      0.2318
brand_LG             -0.0418     0.2812    -0.1485     0.8819     -0.5930      0.5094
brand_Lava           -0.8779     0.4689    -1.8722     0.0612     -1.7970      0.0412
brand_Lenovo         -0.2099     0.3904    -0.5378     0.5907     -0.9751      0.5552
brand_Motorola       -0.3352     0.3873    -0.8655     0.3868     -1.0942      0.4239
brand_Nokia          -0.5411     0.2960    -1.8278     0.0676     -1.1213      0.0391
brand_OnePlus         1.7568     0.4033     4.3564     0.0000      0.9664      2.5472
brand_Oppo            0.2836     0.2403     1.1801     0.2379     -0.1874      0.7547
brand_Realme          0.1667     0.3016     0.5526     0.5805     -0.4245      0.7578
brand_Samsung         0.5160     0.2476     2.0841     0.0371      0.0307      1.0012
brand_Sony           -0.7367     0.3073    -2.3971     0.0165     -1.3390     -0.1343
brand_Tecno          -0.4824     0.2564    -1.8810     0.0600     -0.9850      0.0203
brand_Vivo            0.7380     0.5153     1.4323     0.1521     -0.2719      1.7479
Specs Score           0.0600     0.0068     8.8596     0.0000      0.0467      0.0733
year_of_release      -0.0324     0.0846    -0.3834     0.7014     -0.1983      0.1334
battery_capacity     -0.0004  9.146e-05    -4.1352     0.0000     -0.0006     -0.0002
=====================================================================================
OLSResults, id: 0x16a892f92b0

In [39]:
res_iv2 = IV2SLS(data['log_price'], data[mask_exog2], data['ram'], 
                data['Specs Score']).fit()
print(res_iv2)

                          IV-2SLS Estimation Summary                          
Dep. Variable:              log_price   R-squared:                      0.5754
Estimator:                    IV-2SLS   Adj. R-squared:                 0.5504
No. Observations:                 559   F-statistic:                    3346.3
Date:                Tue, May 09 2023   P-value (F-stat)                0.0000
Time:                        23:06:01   Distribution:                 chi2(31)
Cov. Estimator:                robust                                         
                                                                              
                                 Parameter Estimates                                 
                   Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
-------------------------------------------------------------------------------------
const                -60.803     39.337    -1.5457     0.1222     -137.90      16.295
rating                0.

In [40]:
print(res_iv2.wu_hausman())


Wu-Hausman test of exogeneity
H0: All endogenous variables are exogenous
Statistic: 0.0305
P-value: 0.8614
Distributed: F(1,526)


#### 7.3 battery_capacity - эндогенный

In [31]:
mask_exog_and_instr3 = ['const', 'rating', 'disp_diag', 'camera', 'storage',
               'density', 'Other_oper_syst', 'battery_type_LiPo', 'brand_Apple',
               'brand_BlackBerry', 'brand_Cubot', 'brand_Gionee', 'brand_Google',
               'brand_HTC', 'brand_Honor', 'brand_Huawei', 'brand_Infinix', 'brand_LG',
               'brand_Lava', 'brand_Lenovo', 'brand_Motorola', 'brand_Nokia',
               'brand_OnePlus', 'brand_Oppo', 'brand_Realme', 'brand_Samsung',
               'brand_Sony', 'brand_Tecno', 'brand_Vivo', 'Specs Score', 'year_of_release', 'ram']

mask_exog3 = ['const', 'rating', 'disp_diag', 'camera', 'storage',
               'density', 'Other_oper_syst', 'battery_type_LiPo', 'brand_Apple',
               'brand_BlackBerry', 'brand_Cubot', 'brand_Gionee', 'brand_Google',
               'brand_HTC', 'brand_Honor', 'brand_Huawei', 'brand_Infinix', 'brand_LG',
               'brand_Lava', 'brand_Lenovo', 'brand_Motorola', 'brand_Nokia',
               'brand_OnePlus', 'brand_Oppo', 'brand_Realme', 'brand_Samsung',
               'brand_Sony', 'brand_Tecno', 'brand_Vivo', 'year_of_release', 'ram']

In [41]:
first_step_2SLS3 = IV2SLS(dependent = data['battery_capacity'],  exog = data[mask_exog_and_instr3], endog = None, instruments= None, weights=  None)
first_step_2SLS3 = first_step_2SLS3.fit()
first_step_2SLS3

OLS Estimation Summary                            
==============================================================================
Dep. Variable:       battery_capacity   R-squared:                      0.7142
Estimator:                        OLS   Adj. R-squared:                 0.6974
No. Observations:                 559   F-statistic:                    3654.6
Date:                Tue, May 09 2023   P-value (F-stat)                0.0000
Time:                        23:06:12   Distribution:                 chi2(31)
Cov. Estimator:                robust                                         
                                                                              
                                 Parameter Estimates                                 
=====================================================================================
                   Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
-------------------------------------------------------------------------------------
const             -9.701e+04  3.957e+04    -2.4514     0.0142  -1.746e+05  -1.945e+04
rating               -49.768     28.354    -1.7552     0.0792     -105.34      5.8058
disp_diag             875.09     66.227     13.214     0.0000      745.29      1004.9
camera               -9.1268     3.2102    -2.8430     0.0045     -15.419     -2.8348
storage              -0.9406     0.3403    -2.7639     0.0057     -1.6076     -0.2736
density              -0.0005     0.0009    -0.6090     0.5425     -0.0023      0.0012
Other_oper_syst      -62.768     428.89    -0.1463     0.8836     -903.38      777.84
battery_type_LiPo     184.89     55.067     3.3577     0.0008      76.966      292.82
brand_Apple          -462.62     95.767    -4.8307     0.0000     -650.32     -274.92
brand_BlackBerry      587.26     180.08     3.2612     0.0011      234.32      940.21
brand_Cubot           214.97     355.84     0.6041     0.5458     -482.46      912.41
brand_Gionee          119.97     74.204     1.6168     0.1059     -25.464      265.41
brand_Google         -266.76     147.37    -1.8102     0.0703     -555.59      22.074
brand_HTC            -228.75     93.044    -2.4585     0.0140     -411.11     -46.383
brand_Honor          -568.33     75.497    -7.5279     0.0000     -716.30     -420.36
brand_Huawei         -315.27     70.750    -4.4561     0.0000     -453.94     -176.60
brand_Infinix         35.242     86.888     0.4056     0.6850     -135.06      205.54
brand_LG             -367.76     94.281    -3.9007     0.0001     -552.55     -182.97
brand_Lava           -617.78     221.18    -2.7931     0.0052     -1051.3     -184.27
brand_Lenovo         -409.35     125.88    -3.2519     0.0011     -656.07     -162.63
brand_Motorola        192.70     370.41     0.5202     0.6029     -533.30      918.70
brand_Nokia          -469.61     101.94    -4.6067     0.0000     -669.41     -269.81
brand_OnePlus        -457.03     86.557    -5.2801     0.0000     -626.68     -287.38
brand_Oppo           -186.59     82.806    -2.2533     0.0242     -348.88     -24.291
brand_Realme          325.22     134.48     2.4183     0.0156      61.640      588.79
brand_Samsung        -71.232     86.202    -0.8263     0.4086     -240.18      97.720
brand_Sony           -155.59     137.28    -1.1334     0.2570     -424.65      113.47
brand_Tecno           176.82     131.69     1.3427     0.1794     -81.291      434.94
brand_Vivo            678.76     71.026     9.5565     0.0000      539.55      817.96
Specs Score           10.324     2.5234     4.0916     0.0000      5.3788      15.270
year_of_release       47.270     19.703     2.3991     0.0164      8.6529      85.886
ram                  -33.315     9.2537    -3.6002     0.0003     -51.452     -15.178
=====================================================================================
OLSResults, id: 0x16a896debe0

In [42]:
res_iv3 = IV2SLS(data['log_price'], data[mask_exog3], data['battery_capacity'], 
                data['Specs Score']).fit()
print(res_iv3)

                          IV-2SLS Estimation Summary                          
Dep. Variable:              log_price   R-squared:                      0.5749
Estimator:                    IV-2SLS   Adj. R-squared:                 0.5499
No. Observations:                 559   F-statistic:                    5020.9
Date:                Tue, May 09 2023   P-value (F-stat)                0.0000
Time:                        23:06:20   Distribution:                 chi2(31)
Cov. Estimator:                robust                                         
                                                                              
                                 Parameter Estimates                                 
                   Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
-------------------------------------------------------------------------------------
const                -56.705     46.456    -1.2206     0.2222     -147.76      34.347
rating                0.

In [43]:
print(res_iv3.wu_hausman())


Wu-Hausman test of exogeneity
H0: All endogenous variables are exogenous
Statistic: 0.0305
P-value: 0.8614
Distributed: F(1,526)


Относительно тестов Хаусмана - гипотезы о эндогенности отвергаются, но это может связанно как с невалидностью инструмента, так с эндогенностью регрессора.